### Cell map of the zebrafish embryo using graph abstraction

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy.sparse import csr_matrix
import scanpy as sc

sc.set_figure_params(dpi=200, color_map='viridis')  # low dpi (dots per inch) yields small inline figures
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
results_file = './write/zebrafish_theis.h5ad'

scanpy==1.5.1 anndata==0.7.3 umap==0.4.4 numpy==1.19.0 scipy==1.5.0 pandas==1.0.5 scikit-learn==0.23.1 statsmodels==0.11.1 python-igraph==0.8.2 louvain==0.7.0 leidenalg==0.8.1


In [2]:
pwd


'/Users/reang/Desktop/reang/compbio/notebooks'

In [3]:
var = pd.read_csv('../datasets/data/gene_names.txt', index_col=0, header=None)

In [4]:
cluster_indices = np.genfromtxt('../datasets/data/cell_cluster_indexes.txt', dtype=int)
clusters = pd.Categorical(cluster_indices)
clusters.categories = ['?'] + np.genfromtxt('../datasets/data/cluster_names.txt', dtype=str, delimiter=',').tolist()
obs = pd.DataFrame({'clusters': clusters})

In [5]:
cell_cell_edges = np.genfromtxt('../datasets/data/cell_cell_edges.csv', dtype=int, delimiter=',')
cell_cell_edges -= 1  # our indexing starts with 0 as common in Python, C etc.
from scipy.sparse import coo_matrix
rows = cell_cell_edges[:, 0]
cols = cell_cell_edges[:, 1]
length = len(obs)
ones = np.ones(len(rows), np.uint32)
connectivities = coo_matrix((ones, (rows, cols)), shape=(length, length))
# make sure it's symmsetric
connectivities = connectivities + connectivities.T

In [25]:
uns = {'neighbors': {'connectivities': connectivities.tocsr()}}

In [34]:
adata = sc.AnnData(obs=obs, var=var, uns=uns)
# remove the outliers
adata = adata[adata.obs['clusters'] != '?']

In [37]:
adata = adata

In [38]:
sc.pp.neighbors(adata)

computing neighbors


TypeError: 'NoneType' object is not subscriptable

In [28]:
adata.write(results_file)

TypeError: 'NoneType' object is not subscriptable

In [22]:
adata = sc.read(results_file)

In [23]:
sc.tl.umap(adata)

ValueError: Did not find .uns["neighbors"]. Run `sc.pp.neighbors` first.

In [24]:
sc.tl.draw_graph(adata, maxiter=100)

drawing single-cell graph using layout 'fa'


KeyError: 'No "neighbors" in .uns'